In [1]:
# loading model
from evaluate_detexd_roberta import predict_delicate
from transformers import pipeline

classifier = pipeline("text-classification", model="grammarly/detexd-roberta-base", device=0, batch_size=8)
predict_delicate(classifier, ['hello', 'fuck'])

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


  0%|          | 0/2 [00:00<?, ?it/s]

[False, True]

## Founta dataset

### Data preparation

In [2]:
# you can find the original data here: https://zenodo.org/record/3706866#.Y35QVOzMIZ9

import pandas as pd

df = pd.read_table('founta_data/hatespeech_text_label_vote_RESTRICTED_100K.csv',
                   names=['text', 'label', 'votes'], skiprows=2)
# skipping broken lines

In [3]:
# the dataset got bigger 80k -> 100k
# there's no information whant changed
# the rows are shuffled and ids removed, so we can not restore original dataset
df.label.value_counts().sort_index()

label
abusive    27150
hateful     4965
normal     53851
spam       14029
Name: count, dtype: int64

In [4]:
# removing spam since there are delicate texts there that are not filtered like porn
df = df[df.label != 'spam']

In [5]:
df.label.value_counts().sort_index()

label
abusive    27150
hateful     4965
normal     53851
Name: count, dtype: int64

In [6]:
import datasets
from transformers.pipelines.pt_utils import KeyDataset

preds = predict_delicate(classifier, KeyDataset(datasets.Dataset.from_pandas(df), 'text'))

  0%|          | 0/85966 [00:00<?, ?it/s]

In [7]:
df['real'] = df.label != 'normal'

In [8]:
df['pred'] = preds

In [18]:
df.to_csv('founta_data/preds.csv', index=False)

### Data analysis

In [19]:
df = pd.read_csv('founta_data/preds.csv')

In [9]:
df.real.mean()

0.3735779261568527

In [10]:
from sklearn.metrics import precision_recall_fscore_support
founta_results = precision_recall_fscore_support(y_true=df.real, y_pred=df.pred, average='binary')
founta_results

(0.7626132876614572, 0.6655145570605636, 0.7107630401888895, None)

In [ ]:
# FPs
print('\n------\n'.join(df[df.pred & ~df.real].sample(10, random_state=23543).text))

In [ ]:
# FNs
print('\n------\n'.join(df[~df.pred & df.real].sample(10, random_state=23543).text))

## Basile dataset

### Data preparation

In [11]:
# you can find the original data here: http://hatespeech.di.unito.it/hateval.html
# or here: https://github.com/alisonrib17/SemEval2019-Task5/tree/master/English

df = pd.read_table('basile_data/test_en.tsv')

In [12]:
df.text.str.contains('bitch', case=False).mean()

0.4834224598930481

In [13]:
df['pred'] = predict_delicate(classifier, KeyDataset(datasets.Dataset.from_pandas(df), 'text'))
df['real'] = df.HS == 1

  0%|          | 0/2805 [00:00<?, ?it/s]

In [30]:
df.to_csv('basile_data/preds.csv', index=False)

### Data analysis

In [31]:
df = pd.read_csv('basile_data/preds.csv')

In [14]:
df.real.mean()

0.4206773618538324

In [15]:
df.pred.mean()

0.7875222816399287

In [16]:
basile_results = precision_recall_fscore_support(y_true=df.real, y_pred=df.pred, average='binary')
basile_results

(0.47532820280669985, 0.8898305084745762, 0.6196518146946001, None)

In [ ]:
# FPs
print('\n------\n'.join(df[df.pred & ~df.real].sample(10, random_state=23543).text))

In [ ]:
# FNs
print('\n------\n'.join(df[~df.pred & df.real].sample(10, random_state=23543).text))

In [19]:
pd.DataFrame([founta_results, basile_results],
             columns=['precision', 'recall', 'f-score', '_'],
             index=['Founta', 'Basile']
            ).iloc[:, :3].style.format('{:.1%}'.format)

,precision,recall,f-score
Founta,76.3%,66.6%,71.1%
Basile,47.5%,89.0%,62.0%


### Restoring precision and recall

In [1]:
def positive_class_metrics(positive_class_percent, macro_f1, accuracy):
    def f1_macro_fn(tp):
        tn = accuracy - tp
        fp = negative_class_percent - tn
        fn = positive_class_percent - tp
        p1 = tp / (tp + fp + 1e-100)
        r1 = tp / (tp + fn + 1e-100)
        p0 = tn / (tn + fn + 1e-100)
        r0 = tn / (tn + fp + 1e-100)
        f1_fn = p0 * r0 / (p0 + r0 + 1e-100) + p1 * r1 / (p1 + r1 + 1e-100)
        return f1_fn, p1, r1

    negative_class_percent = 1 - positive_class_percent
    n = 1000
    diff, tp_opt = min((abs(f1_macro_fn(tp / n)[0] - macro_f1), tp / n) for tp in range(n + 1))
    _, p1, r1 = f1_macro_fn(tp_opt)
    f1 = 2 / (1 / p1 + 1 / r1)
    print(f'precision: {p1:.1%}')
    print(f'recall: {r1:.1%}')
    print(f'f1: {f1:.1%}')


positive_class_metrics(
    positive_class_percent=0.4206773618538324,
    accuracy=0.65,
    macro_f1=0.651)

precision: 56.1%
recall: 77.3%
f1: 65.0%
